模型参数的访问、初始化和共享

In [1]:
import torch
from torch import nn
from torch.nn import init
net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1)) #
print(net)
X = torch.rand(2, 4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


1.访问模型参数

In [3]:
#访问模型参数：named_parameters()
print(type(net.named_parameters()))  # 类型检查
for name, param in net.named_parameters():
    print(name, param.size())  # 打印名称和形状
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))  # 单层参数 + 类型
#MyModel示例：Parameter vs. Tensor
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))  # Parameter（注册）
        self.weight2 = torch.rand(20, 20)  # 普通Tensor（不注册）
    def forward(self, x):
        pass  # 空，焦点参数

n = MyModel()
for name, param in n.named_parameters():
    print(name)  # 只print weight1
#访问data和grad
# 访问data和grad
weight_0 = list(net[0].named_parameters())[0]  # 取第一个参数 (name, param) 元组
print(weight_0[1].data)  # 数据值 - 修正：使用[1]获取参数对象
print(weight_0[1].grad)  # 梯度 (初始None) - 修正：使用[1]获取参数对象
Y.backward()  # 反向传播
print(weight_0[1].grad)  # 梯度值 - 修正：使用[1]获取参数对象

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])
weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>
weight1
tensor([[ 0.2695, -0.1191,  0.1470,  0.4575],
        [ 0.3044, -0.2531,  0.1825,  0.0920],
        [-0.1740, -0.1391,  0.1717, -0.1503]])
None
tensor([[ 0.1623,  0.1111,  0.1144,  0.1902],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.1541, -0.1055, -0.1086, -0.1807]])


2.初始化模型参数

In [4]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)  # 初始化为N(0,0.01)
print(name, param.data)  # 打印name和值

for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)  # 初始化为常数0
print(name, param.data)  # 打印name和值

2.bias tensor([-0.0982])
2.bias tensor([0.])


3.⾃自定义初始化⽅方法

In [5]:
def normal_(tensor, mean=0, std=1):
    with torch.no_grad():
        return tensor.normal_(mean, std)
    
def init_weight_(tensor):
    with torch.no_grad():
        # 第一步：在[-10, 10]范围内均匀分布初始化
        tensor.uniform_(-10, 10)
        # 第二步：将绝对值小于5的值置为0
        tensor *= (tensor.abs() >= 5).float()
# 遍历网络所有参数
for name, param in net.named_parameters():
    if 'weight' in name:  # 只对权重参数应用
        init_weight_(param)
        print(name, param.data)

for name, param in net.named_parameters():
    if 'bias' in name:  # 对偏置参数加1
        param.data += 1  # 直接操作.data，不影响计算图
        print(name, param.data)

0.weight tensor([[ 0.0000,  9.1267,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -6.9104, -0.0000],
        [ 0.0000,  6.4669, -0.0000, -5.5608]])
2.weight tensor([[ 0.0000, -9.0757,  0.0000]])
0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


4.共享模型参数

In [6]:
import torch
import torch.nn as nn
from torch.nn import init

# 创建单个线性层实例
linear = nn.Linear(1, 1, bias=False)
# 在Sequential中重复使用同一个实例
net = nn.Sequential(linear, linear)

print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)
# 输出: weight tensor([[3.]])

# 验证是同一个对象
print(id(net[0]) == id(net[1]))          # True - 是同一个层实例
print(id(net[0].weight) == id(net[1].weight))  # True - 是同一个参数张量

x = torch.ones(1, 1)
y = net(x).sum()
print(y)  # 计算过程: x -> linear -> linear -> sum()
# 输出: tensor(9., grad_fn=<SumBackward0>)
# 计算过程: 3*1 = 3 -> 3*3 = 9

y.backward()
print(net[0].weight.grad)  # 单次梯度是3，两次所以就是6
# 输出: tensor([[6.]])

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])
True
True
tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
